In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
pip install transformers sentence-transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer, models
from datasets import load_dataset
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm, trange
import time
import datetime
import random
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
dataset = load_dataset("stsb_multi_mt", name="ru")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/721k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/158k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/209k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [6]:
print(type(dataset))

<class 'datasets.dataset_dict.DatasetDict'>


In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 5749
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1379
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1500
    })
})


In [8]:
df = dataset['train'].to_pandas()

In [9]:
df.head()

,sentence1,sentence2,similarity_score
0,Самолет взлетает.,Взлетает самолет.,5.00
1,Человек играет на большой флейте.,Человек играет на флейте.,3.80
2,Мужчина разбрасывает сыр на пиццу.,Мужчина разбрасывает измельченный сыр на варен...,3.80
3,Трое мужчин играют в шахматы.,Двое мужчин играют в шахматы.,2.60
4,Мужчина играет на виолончели.,"Человек, сидящий на виолончели, играет на виол...",4.25


In [10]:
df.shape

(5749, 3)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
file_path = '/content/drive/My Drive/df_res.csv'

In [13]:
df_res = pd.read_csv(file_path)

In [14]:
df_res.head()

,sentence1,sentence2,similarity_score
0,big data,большие данные,5.0
1,machine learning,машинное обучение,5.0
2,deep learning,глубокое обучение,5.0
3,artificial intelligence,искусственный интеллект,5.0
4,ai,ии,5.0


In [15]:
df_res_m = Dataset.from_pandas(df_res)

In [16]:
df_res_m

Dataset({
    features: ['sentence1', 'sentence2', 'similarity_score'],
    num_rows: 679
})

In [17]:
 # Объединение нового датасета с существующим тренировочным датасетом
combined_train = Dataset.from_dict({
       'sentence1': dataset['train']['sentence1'] + df_res_m['sentence1'],
       'sentence2': dataset['train']['sentence2'] + df_res_m['sentence2'],
       'similarity_score': dataset['train']['similarity_score'] + df_res_m['similarity_score']
   })


   # Обновление DatasetDict
dataset['train'] = combined_train

In [18]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 6428
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1379
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1500
    })
})


In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [19]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
class STSBDataset(torch.utils.data.Dataset):

    def __init__(self, dataset):
        # Normalize the similarity scores in the dataset
        similarity_scores = [i['similarity_score'] for i in dataset]
        self.normalized_similarity_scores = [i/5.0 for i in similarity_scores]               #Как раз таки здесь!
        self.first_sentences = [i['sentence1'] for i in dataset]
        self.second_sentences = [i['sentence2'] for i in dataset]
        self.concatenated_sentences = [[str(x), str(y)] for x,y in   zip(self.first_sentences, self.second_sentences)]

    def __len__(self):
        return len(self.concatenated_sentences)

    def get_batch_labels(self, idx):
        return torch.tensor(self.normalized_similarity_scores[idx])

    def get_batch_texts(self, idx):
        return tokenizer(self.concatenated_sentences[idx], padding='max_length', max_length=128, truncation=True, return_tensors="pt")

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [21]:
def collate_fn(texts):
    input_ids = texts['input_ids']
    attention_masks = texts['attention_mask']
    features = [{'input_ids': input_id, 'attention_mask': attention_mask}
                for input_id, attention_mask in zip(input_ids, attention_masks)]
    return features

In [22]:
class BertForSTS(torch.nn.Module):

    def __init__(self):
        super(BertForSTS, self).__init__()
        self.bert = models.Transformer('cointegrated/rubert-tiny', max_seq_length=128)
        #self.bert = AutoModel.from_pretrained("cointegrated/rubert-tiny")
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])

    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output

In [23]:
model = BertForSTS()
model.to(device)

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

BertForSTS(
  (bert): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (pooling_layer): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (sts_bert): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
)

In [24]:
class CosineSimilarityLoss(torch.nn.Module):

    def __init__(self,  loss_fn=torch.nn.MSELoss(), transform_fn=torch.nn.Identity()):
        super(CosineSimilarityLoss, self).__init__()
        self.loss_fn = loss_fn
        self.transform_fn = transform_fn
        self.cos_similarity = torch.nn.CosineSimilarity(dim=1)

    def forward(self, inputs, labels):
        emb_1 = torch.stack([inp[0] for inp in inputs])
        emb_2 = torch.stack([inp[1] for inp in inputs])
        outputs = self.transform_fn(self.cos_similarity(emb_1, emb_2))
        return self.loss_fn(outputs, labels.squeeze())

In [25]:
train_ds = STSBDataset(dataset['train'])
val_ds = STSBDataset(dataset['dev'])

# Create a 90-10 train-validation split.
train_size = len(train_ds)
val_size = len(val_ds)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

6,428 training samples
1,500 validation samples


In [26]:
batch_size = 8

train_dataloader = DataLoader(
            train_ds,  # The training samples.
            num_workers = 2,
            batch_size = batch_size, # Use this batch size.
            shuffle=True # Select samples randomly for each batch
        )

validation_dataloader = DataLoader(
            val_ds,
            num_workers = 2,
            batch_size = batch_size # Use the same batch size
        )

In [27]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-6)
epochs = 100
# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)



In [28]:
def format_time(elapsed):
    """
    Принимает время в секундах и возвращает строку в формате hh:mm:ss.
    """
    elapsed_rounded = int(round((elapsed)))
    return str(time.strftime("%H:%M:%S", time.gmtime(elapsed_rounded)))

In [29]:
def train():
  seed_val = 42
  criterion = CosineSimilarityLoss()
  criterion = criterion.cuda()
  random.seed(seed_val)
  torch.manual_seed(seed_val)
  # We'll store a number of quantities such as training and validation loss,
  # validation accuracy, and timings.
  training_stats = []
  total_t0 = time.time()
  for epoch_i in range(0, epochs):
      t0 = time.time()
      total_train_loss = 0
      model.train()
      # For each batch of training data...
      for train_data, train_label in tqdm(train_dataloader):
          train_data['input_ids'] = train_data['input_ids'].to(device)
          train_data['attention_mask'] = train_data['attention_mask'].to(device)
          train_data = collate_fn(train_data)
          model.zero_grad()
          output = [model(feature) for feature in train_data]
          loss = criterion(output, train_label.to(device))
          total_train_loss += loss.item()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)
      t0 = time.time()
      model.eval()
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0
      # Evaluate data for one epoch
      for val_data, val_label in tqdm(validation_dataloader):
          val_data['input_ids'] = val_data['input_ids'].to(device)
          val_data['attention_mask'] = val_data['attention_mask'].to(device)
          val_data = collate_fn(val_data)
          with torch.no_grad():
              output = [model(feature) for feature in val_data]
          loss = criterion(output, val_label.to(device))
          total_eval_loss += loss.item()
      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )
  return model, training_stats

# Launch the training
model, training_stats = train()

  0%|          | 0/804 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/804 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/804 [00:00<?, ?it/s]

...

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/804 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/804 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

In [30]:
# Create a DataFrame from our training statistics
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index
df_stats = df_stats.set_index('epoch')

# Display the table
df_stats

,Training Loss,Valid. Loss,Training Time,Validation Time
epoch,,,,
1,0.091449,0.077654,00:01:01,00:00:06
2,0.064917,0.060318,00:00:59,00:00:07
3,0.055118,0.053805,00:00:59,00:00:06
4,0.051515,0.050898,00:00:59,00:00:07
5,0.049311,0.049198,00:00:58,00:00:08
...,...,...,...,...
96,0.023534,0.039674,00:00:59,00:00:08
97,0.023444,0.039674,00:00:59,00:00:06
98,0.023485,0.039671,00:00:59,00:00:07


In [31]:
# load the test set
test_dataset = load_dataset("stsb_multi_mt", name="ru", split="test")

# Prepare the data
first_sent = [i['sentence1'] for i in test_dataset]
second_sent = [i['sentence2'] for i in test_dataset]
full_text = [[str(x), str(y)] for x,y in zip(first_sent, second_sent)]

model.eval()

def predict_similarity(sentence_pair):
  test_input = tokenizer(sentence_pair, padding='max_length', max_length = 128, truncation=True, return_tensors="pt").to(device)
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']
  output = model(test_input)
  sim = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()
  return sim

In [32]:
example_1 = full_text[100]
print(f"Sentence 1: {example_1[0]}")
print(f"Sentence 2: {example_1[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_1), 2)}")

Sentence 1: Кошка гуляет по дому.
Sentence 2: Женщина чистит картошку.
Predicted similarity score: 0.13


In [33]:
example_2 = full_text[130]
print(f"Sentence 1: {example_2[0]}")
print(f"Sentence 2: {example_2[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_2), 2)}")

Sentence 1: Двое мужчин играют в футбол.
Sentence 2: Двое мужчин занимаются футболом.
Predicted similarity score: 0.87


In [34]:
example_3 = full_text[812]
print(f"Sentence 1: {example_3[0]}")
print(f"Sentence 2: {example_3[1]}")
print(f"Predicted similarity score: {round(predict_similarity(example_3), 2)}")

Sentence 1: Это зависит от ситуации.
Sentence 2: Это варьируется в зависимости от учреждения.
Predicted similarity score: 0.6


In [35]:
sentence_1 = 'жуков сергей владимирович data scientist junior опыт работы otus https otus ru профессия data scientist обучение изучены теоретические основы математики для data science практика exploratory data analysis eda preprocessing feature engineering и визуализация данных классификация влияние коэффициентов на целевую переменную knn logistic regressioncv линейная регрессия модели linearregression ridge cv lassocv и elasticnetcv gradient boosting sklearn xgboost catboost lig htgbm кластеризация k means hierarhical и dbscan сжатие признакового пространства pca umap tsne anomaly detection подходы rule citiblock mode l dbscan oneclasssvm isolationforest нейронные сети на pytorch переобучение и регуляризация l l dropout batchnorm deep learning автокодировщики и автокодирование обучил многослойного и шумного автокодировщика временные ряды применил модель sarima linear regression gradient boosting для прогнозирования вр сверточные нейронные сети модели resnet mobile net efficientnet dencenet inception v для задачи классификации аугментация данных albumentations и torchvision object detection модель из detectron метрика точности coco map рекомендательные системы svd алгоритм библиотеки surprise парсинг и nlp beautifulsoup лемматизация стемминг snowballstemmer отсев стоп слов и tf idf преобразования rl q learning для поиска оптимальной стратегии образование казанский национальный исследовательский технологический университет магистр информатика и вычислительная техника диплом с отличием дипломный проект разработка информационной систем ы по учету химических реагентов казанский национальный исследовательский технологический университет институт нефти и нефтехимии специалист химическая технология органических веществ о себе после получения первого высшего образования устроился в нефтесервисную компанию миррико где продолжаю работать прошел карьерный путь от лаборанта до ведущего инженера к моим функциональным обязанностям относятся как исследовательская деятельность так и установка технологических составов на скважинах по ряду причин задумался о смене рода деятельности всегда интересовался it поэтому решил получить второе высшее в данной сфере во время обучения больше всего нравилось все что связано с большими данными решив углубить свои знания в данной области успешно прошел курс по data science в otus контакты телефон электронная почта zhukovs v yandex ru telegram https t me zhukovetc портфолио https github com zhukovserg otushw навыки python ооп sql git numpy gym pandas sklearn matplotlib seaborn pytorch scipy surprise docker языки русский родной английский b средний'

In [36]:
sentence_2 = sentence_1

In [37]:
sentence_pair = sentence_1, sentence_2
similarity_score = predict_similarity(sentence_pair)
print(f"Predicted similarity score: {similarity_score}")

Predicted similarity score: 1.0


In [38]:
sentence_1 = 'жуков сергей владимирович data scientist junior опыт работы otus https otus ru профессия data scientist обучение изучены теоретические основы математики для data science практика exploratory data analysis eda preprocessing feature engineering и визуализация данных классификация влияние коэффициентов на целевую переменную knn logistic regressioncv линейная регрессия модели linearregression ridge cv lassocv и elasticnetcv gradient boosting sklearn xgboost catboost lig htgbm кластеризация k means hierarhical и dbscan сжатие признакового пространства pca umap tsne anomaly detection подходы rule citiblock mode l dbscan oneclasssvm isolationforest нейронные сети на pytorch переобучение и регуляризация l l dropout batchnorm deep learning автокодировщики и автокодирование обучил многослойного и шумного автокодировщика временные ряды применил модель sarima linear regression gradient boosting для прогнозирования вр сверточные нейронные сети модели resnet mobile net efficientnet dencenet inception v для задачи классификации аугментация данных albumentations и torchvision object detection модель из detectron метрика точности coco map рекомендательные системы svd алгоритм библиотеки surprise парсинг и nlp beautifulsoup лемматизация стемминг snowballstemmer отсев стоп слов и tf idf преобразования rl q learning для поиска оптимальной стратегии образование казанский национальный исследовательский технологический университет магистр информатика и вычислительная техника диплом с отличием дипломный проект разработка информационной систем ы по учету химических реагентов казанский национальный исследовательский технологический университет институт нефти и нефтехимии специалист химическая технология органических веществ о себе после получения первого высшего образования устроился в нефтесервисную компанию миррико где продолжаю работать прошел карьерный путь от лаборанта до ведущего инженера к моим функциональным обязанностям относятся как исследовательская деятельность так и установка технологических составов на скважинах по ряду причин задумался о смене рода деятельности всегда интересовался it поэтому решил получить второе высшее в данной сфере во время обучения больше всего нравилось все что связано с большими данными решив углубить свои знания в данной области успешно прошел курс по data science в otus контакты телефон электронная почта zhukovs v yandex ru telegram https t me zhukovetc портфолио https github com zhukovserg otushw навыки python ооп sql git numpy gym pandas sklearn matplotlib seaborn pytorch scipy surprise docker языки русский родной английский b средний'

In [39]:
sentence_2 =  'жуков сергей владимирович ученый по данным начинающий опыт работы otus https otus ru профессия ученый по данным обучение изучены теоретические основы математики для наука о данных практика exploratory data analysis eda preprocessing feature engineering и визуализация данных классификация влияние коэффициентов на целевую переменную knn logistic regressioncv линейная регрессия модели linearregression ridge cv lassocv и elasticnetcv gradient boosting sklearn xgboost catboost lig htgbm кластеризация k means hierarhical и dbscan сжатие признакового пространства pca umap tsne anomaly detection подходы rule citiblock mode l dbscan oneclasssvm isolationforest нейронные сети на pytorch переобучение и регуляризация l l dropout batchnorm deep learning автокодировщики и автокодирование обучил многослойного и шумного автокодировщика временные ряды применил модель sarima linear regression gradient boosting для прогнозирования вр сверточные нейронные сети модели resnet mobile net efficientnet dencenet inception v для задачи классификации аугментация данных albumentations и torchvision object detection модель из detectron метрика точности coco map рекомендательные системы svd алгоритм библиотеки surprise парсинг и nlp beautifulsoup лемматизация стемминг snowballstemmer отсев стоп слов и tf idf преобразования rl q learning для поиска оптимальной стратегии образование казанский национальный исследовательский технологический университет магистр информатика и вычислительная техника диплом с отличием дипломный проект разработка информационной систем ы по учету химических реагентов казанский национальный исследовательский технологический университет институт нефти и нефтехимии специалист химическая технология органических веществ о себе после получения первого высшего образования устроился в нефтесервисную компанию миррико где продолжаю работать прошел карьерный путь от лаборанта до ведущего инженера к моим функциональным обязанностям относятся как исследовательская деятельность так и установка технологических составов на скважинах по ряду причин задумался о смене рода деятельности всегда интересовался it поэтому решил получить второе высшее в данной сфере во время обучения больше всего нравилось все что связано с большими данными решив углубить свои знания в данной области успешно прошел курс по data science в otus контакты телефон электронная почта zhukovs v yandex ru telegram https t me zhukovetc портфолио https github com zhukovserg otushw навыки python ооп sql git numpy gym pandas sklearn matplotlib seaborn pytorch scipy surprise docker языки русский родной английский b средний'

In [40]:
sentence_pair = sentence_1, sentence_2
similarity_score = predict_similarity(sentence_pair)
print(f"Predicted similarity score: {similarity_score}")

Predicted similarity score: 0.9849017858505249


In [41]:
sentence_1 = 'python разработчик'

In [42]:
sentence_2 = 'python developer'

In [43]:
sentence_pair = sentence_1, sentence_2
similarity_score = predict_similarity(sentence_pair)
print(f"Predicted similarity score: {similarity_score}")

Predicted similarity score: 0.9848189353942871


In [49]:
torch.save(model.state_dict(), 'bert_sts_model.pth')